In [2]:
# !wget -nc -P data/ https://archive.org/download/stackexchange/ai.stackexchange.com.7z/Posts.xml

--2022-10-03 16:07:54--  https://archive.org/download/stackexchange/ai.stackexchange.com.7z/Posts.xml
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia600107.us.archive.org/view_archive.php?archive=/27/items/stackexchange/ai.stackexchange.com.7z&file=Posts.xml [following]
--2022-10-03 16:07:55--  https://ia600107.us.archive.org/view_archive.php?archive=/27/items/stackexchange/ai.stackexchange.com.7z&file=Posts.xml
Resolving ia600107.us.archive.org (ia600107.us.archive.org)... 207.241.227.247
Connecting to ia600107.us.archive.org (ia600107.us.archive.org)|207.241.227.247|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/xml]
Saving to: ‘data/Posts.xml’

Posts.xml               [    <=>             ]  37.79M  3.27MB/s    in 8.9s    

2022-10-03 16:08:04 (4.27 MB/s) - ‘data/Posts.xml’ saved [39630077]



In [1]:
import pyspark as ps

In [2]:
sc = ps.SparkContext()

In [3]:
sc.master

'local[*]'

In [4]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [5]:
posts = sc.textFile('data/Posts.xml')

In [6]:
posts.take(5)

['<?xml version="1.0" encoding="utf-8"?>',
 '<posts>',
 '  <row Id="1" PostTypeId="1" AcceptedAnswerId="15" CreationDate="2010-07-19T19:12:12.510" Score="48" ViewCount="4280" Body="&lt;p&gt;How should I elicit prior distributions from experts when fitting a Bayesian model?&lt;/p&gt;&#xA;" OwnerUserId="8" LastActivityDate="2020-11-05T09:44:51.710" Title="Eliciting priors from experts" Tags="&lt;bayesian&gt;&lt;prior&gt;&lt;elicitation&gt;" AnswerCount="6" CommentCount="1" FavoriteCount="32" ContentLicense="CC BY-SA 2.5" />',
 '  <row Id="2" PostTypeId="1" AcceptedAnswerId="59" CreationDate="2010-07-19T19:12:57.157" Score="34" ViewCount="32048" Body="&lt;p&gt;In many different statistical methods there is an &quot;assumption of normality&quot;.  What is &quot;normality&quot; and how do I know if there is normality?&lt;/p&gt;&#xA;" OwnerUserId="24" LastEditorDisplayName="user88" LastEditDate="2010-08-07T17:56:44.800" LastActivityDate="2017-11-22T12:15:07.030" Title="What is normality?" Ta

In [7]:
import xml.etree.ElementTree as etree

In [8]:
element = etree.fromstring(posts.take(3)[2]) 

In [9]:
element

<Element 'row' at 0x7fa2c6b41d60>

In [10]:
element.items()

[('Id', '1'),
 ('PostTypeId', '1'),
 ('AcceptedAnswerId', '15'),
 ('CreationDate', '2010-07-19T19:12:12.510'),
 ('Score', '48'),
 ('ViewCount', '4280'),
 ('Body',
  '<p>How should I elicit prior distributions from experts when fitting a Bayesian model?</p>\n'),
 ('OwnerUserId', '8'),
 ('LastActivityDate', '2020-11-05T09:44:51.710'),
 ('Title', 'Eliciting priors from experts'),
 ('Tags', '<bayesian><prior><elicitation>'),
 ('AnswerCount', '6'),
 ('CommentCount', '1'),
 ('FavoriteCount', '32'),
 ('ContentLicense', 'CC BY-SA 2.5')]

In [ ]:
# filter valid row XML
# convert data into proper types
# convert RDD into DataFrame

In [11]:
def validate_post(xml):
    try:
        row = etree.fromstring(xml)
        if row.get('Id'):
            return True
    except Exception as e:
        print(e)
        return False

In [12]:
posts.filter(validate_post)

PythonRDD[4] at RDD at PythonRDD.scala:53

In [13]:
%%time
posts.filter(validate_post).count()

CPU times: user 7 ms, sys: 17.3 ms, total: 24.3 ms
Wall time: 12.5 s


340838

In [15]:
from datetime import datetime

schema = {
    'Id': int,
    'PostTypeId': int,
    'AcceptedAnswerId': int,
    'CreationDate': datetime.fromisoformat,
    'Score': int,
    'ViewCount': int,
    'Body': str,
    'OwnerUserId': int,
    'LastActivityDate': datetime.fromisoformat,
    'Title': str,
    'Tags': str,
    'AnswerCount': int,
    'CommentCount': int,
    'FavoriteCount': int,
    'ContentLicense': str
}

In [16]:
parsed = posts.filter(validate_post).map(lambda p: { k: schema[k](v) for (k, v) in etree.fromstring(p).items() if k in schema })

In [17]:
parsed.filter(lambda p: 'Title' in p).map(lambda p: p['Title']).take(12)

['Eliciting priors from experts',
 'What is normality?',
 'What are some valuable Statistical Analysis open source projects?',
 'Assessing the significance of differences in distributions',
 'The Two Cultures: statistics vs. machine learning?',
 'Locating freely available data samples',
 'So how many staticians *does* it take to screw in a lightbulb?',
 'Under what conditions should Likert scales be used as ordinal or interval data?',
 'Multivariate Interpolation Approaches',
 'How can I adapt ANOVA for binary data?',
 'Forecasting demographic census',
 'Bayesian and frequentist reasoning in plain English']

In [18]:
print(parsed.toDebugString().decode('ascii'))

(19) PythonRDD[7] at RDD at PythonRDD.scala:53 []
 |   data/Posts.xml MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 []
 |   data/Posts.xml HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 []


In [19]:
parsed.take(1)

[{'Id': 1,
  'PostTypeId': 1,
  'AcceptedAnswerId': 15,
  'CreationDate': datetime.datetime(2010, 7, 19, 19, 12, 12, 510000),
  'Score': 48,
  'ViewCount': 4280,
  'Body': '<p>How should I elicit prior distributions from experts when fitting a Bayesian model?</p>\n',
  'OwnerUserId': 8,
  'LastActivityDate': datetime.datetime(2020, 11, 5, 9, 44, 51, 710000),
  'Title': 'Eliciting priors from experts',
  'Tags': '<bayesian><prior><elicitation>',
  'AnswerCount': 6,
  'CommentCount': 1,
  'FavoriteCount': 32,
  'ContentLicense': 'CC BY-SA 2.5'}]

In [20]:
parsed.count()

340838

In [21]:
%%time
parsed.count()

CPU times: user 18.6 ms, sys: 13 ms, total: 31.5 ms
Wall time: 20.9 s


340838

In [22]:
parsed.cache()

PythonRDD[7] at RDD at PythonRDD.scala:53

In [25]:
print(parsed.toDebugString().decode('ascii'))

(19) PythonRDD[7] at RDD at PythonRDD.scala:53 [Memory Serialized 1x Replicated]
 |        CachedPartitions: 19; MemorySize: 263.5 MiB; DiskSize: 0.0 B
 |   data/Posts.xml MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]
 |   data/Posts.xml HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]


In [30]:
%%time
parsed.count()

CPU times: user 4.74 ms, sys: 11.9 ms, total: 16.6 ms
Wall time: 1.41 s


340838

In [31]:
print(parsed.toDebugString().decode('ascii'))

(19) PythonRDD[7] at RDD at PythonRDD.scala:53 [Memory Serialized 1x Replicated]
 |        CachedPartitions: 19; MemorySize: 263.5 MiB; DiskSize: 0.0 B
 |   data/Posts.xml MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]
 |   data/Posts.xml HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 [Memory Serialized 1x Replicated]


In [32]:
%%time
parsed.count()

CPU times: user 7.19 ms, sys: 18.5 ms, total: 25.7 ms
Wall time: 1.73 s


340838

In [33]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [34]:
spark

In [35]:
parsed

PythonRDD[7] at RDD at PythonRDD.scala:53

In [41]:
# Can create a data frame from a RDD, Hive table, or Spark data source
df = spark.createDataFrame(parsed)

In [ ]:
df.write.orc('data/dataframe.orc')

In [40]:
orc = spark.read.orc('data/dataframe.orc')

In [43]:
df.columns

['AcceptedAnswerId',
 'AnswerCount',
 'Body',
 'CommentCount',
 'ContentLicense',
 'CreationDate',
 'FavoriteCount',
 'Id',
 'LastActivityDate',
 'OwnerUserId',
 'PostTypeId',
 'Score',
 'Tags',
 'Title',
 'ViewCount']

In [45]:
df.select('Title', 'Body', 'Score', 'AcceptedAnswerId').show(100)

+--------------------+--------------------+-----+----------------+
|               Title|                Body|Score|AcceptedAnswerId|
+--------------------+--------------------+-----+----------------+
|Eliciting priors ...|<p>How should I e...|   48|              15|
|  What is normality?|<p>In many differ...|   34|              59|
|What are some val...|<p>What are some ...|   71|               5|
|Assessing the sig...|<p>I have two gro...|   23|             135|
|                null|<p>The R-project<...|   90|            null|
|The Two Cultures:...|<p>Last year, I r...|  458|            null|
|Locating freely a...|<p>I've been work...|  102|              18|
|So how many stati...|<p>Sorry, but the...|    0|            null|
|                null|<p><a href="http:...|   14|            null|
|Under what condit...|<p>Many studies i...|   57|            1887|
|Multivariate Inte...|<p>Is there a goo...|    4|            1201|
|                null|<p>See my respons...|   24|            n

In [46]:
df.printSchema()

root
 |-- AcceptedAnswerId: long (nullable = true)
 |-- AnswerCount: long (nullable = true)
 |-- Body: string (nullable = true)
 |-- CommentCount: long (nullable = true)
 |-- ContentLicense: string (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- FavoriteCount: long (nullable = true)
 |-- Id: long (nullable = true)
 |-- LastActivityDate: timestamp (nullable = true)
 |-- OwnerUserId: long (nullable = true)
 |-- PostTypeId: long (nullable = true)
 |-- Score: long (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- ViewCount: long (nullable = true)



In [47]:
df.head(3)

[Row(AcceptedAnswerId=15, AnswerCount=6, Body='<p>How should I elicit prior distributions from experts when fitting a Bayesian model?</p>\n', CommentCount=1, ContentLicense='CC BY-SA 2.5', CreationDate=datetime.datetime(2010, 7, 19, 19, 12, 12, 510000), FavoriteCount=32, Id=1, LastActivityDate=datetime.datetime(2020, 11, 5, 9, 44, 51, 710000), OwnerUserId=8, PostTypeId=1, Score=48, Tags='<bayesian><prior><elicitation>', Title='Eliciting priors from experts', ViewCount=4280),
 Row(AcceptedAnswerId=59, AnswerCount=7, Body='<p>In many different statistical methods there is an "assumption of normality".  What is "normality" and how do I know if there is normality?</p>\n', CommentCount=1, ContentLicense='CC BY-SA 2.5', CreationDate=datetime.datetime(2010, 7, 19, 19, 12, 57, 157000), FavoriteCount=11, Id=2, LastActivityDate=datetime.datetime(2017, 11, 22, 12, 15, 7, 30000), OwnerUserId=24, PostTypeId=1, Score=34, Tags='<distributions><normality-assumption>', Title='What is normality?', ViewC

In [48]:
df.tail(3)

[Row(AcceptedAnswerId=None, AnswerCount=0, Body="<p>I'm looking for an example of a Python implementation of the Terry-Hoeffding Normal Scores test. I have found other similar tests, but not this one.</p>\n", CommentCount=0, ContentLicense='CC BY-SA 4.0', CreationDate=datetime.datetime(2020, 12, 6, 4, 0, 57, 727000), FavoriteCount=None, Id=499531, LastActivityDate=datetime.datetime(2020, 12, 6, 4, 0, 57, 727000), OwnerUserId=304629, PostTypeId=1, Score=0, Tags='<python>', Title='Python Terry-Hoeffding Test', ViewCount=3),
 Row(AcceptedAnswerId=None, AnswerCount=0, Body='<p>I am attending a course on &quot;Introduction to Machine Learning&quot; where a large portion of this course to my surprise has <strong>probabilistic</strong> approach to machine learning. Like Probabilistic Approach to Linear and logistic regression and thereby trying to find the optimal weights using MLE, MAP or Bayesian. But I am not able to understand what is its importance? We can do all this using non-probabili

In [57]:
df.select('Id', 'Body').show()

+---+--------------------+
| Id|                Body|
+---+--------------------+
|  1|<p>How should I e...|
|  2|<p>In many differ...|
|  3|<p>What are some ...|
|  4|<p>I have two gro...|
|  5|<p>The R-project<...|
|  6|<p>Last year, I r...|
|  7|<p>I've been work...|
|  8|<p>Sorry, but the...|
|  9|<p><a href="http:...|
| 10|<p>Many studies i...|
| 11|<p>Is there a goo...|
| 12|<p>See my respons...|
| 13|<p>Machine Learni...|
| 14|<p>I second that ...|
| 15|<p>John Cook give...|
| 16|<p>Two projects s...|
| 17|<p>I have four co...|
| 18|<p>Also see the U...|
| 19|<p><a href="http:...|
| 20|<p>The assumption...|
+---+--------------------+
only showing top 20 rows



In [58]:
sample = df.sample(fraction=0.01)

In [59]:
sample.select('Id', 'Body').show()

+----+--------------------+
|  Id|                Body|
+----+--------------------+
|  24|<p>For doing a va...|
| 351|<p><a href="http:...|
| 387|<p>The following ...|
| 477|<p>I assume your ...|
| 485|<p>A question pre...|
| 554|<p>I like to demo...|
| 559|<p>The general id...|
|1061|<pre><code>&gt; -...|
|1084|<p>I'm trying to ...|
|1093|<p>I'm looking fo...|
|1119|<p>In addition to...|
|1255|<p>I'm confused: ...|
|1352|<p>In an average ...|
|1374|<p>How many stati...|
|1376|<p>I am looking f...|
|1392|<p>It means that ...|
|1396|<p>You can probab...|
|1631|<p>A z-table give...|
|1639|<p>The t-test wit...|
|1862|<p>I've been stru...|
+----+--------------------+
only showing top 20 rows



In [60]:
pd_df = sample.toPandas()

In [51]:
type(pd_df)

pandas.core.frame.DataFrame

In [54]:
pd_df.describe()

,AcceptedAnswerId,AnswerCount,CommentCount,FavoriteCount,Id,OwnerUserId,PostTypeId,Score,ViewCount
count,570.000000,1745.000000,3429.000000,558.000000,3429.000000,3384.000000,3429.000000,3429.000000,1745.000000
mean,235758.105263,0.984527,1.812773,4.261649,249908.493438,93460.004728,1.509186,3.183727,1907.010888
std,147710.933571,1.203631,2.687203,22.388755,150012.888632,87751.697728,0.561545,13.170149,13722.609608
min,858.000000,0.000000,0.000000,0.000000,36.000000,-1.000000,1.000000,-9.000000,3.000000
25%,99660.500000,0.000000,0.000000,1.000000,116020.000000,14076.000000,1.000000,1.000000,54.000000
50%,231870.500000,1.000000,1.000000,1.000000,251161.000000,66379.000000,1.000000,1.000000,179.000000
75%,364145.250000,1.000000,3.000000,3.000000,383542.000000,158387.750000,2.000000,3.000000,807.000000
max,498470.000000,28.000000,28.000000,465.000000,499466.000000,304580.000000,5.000000,551.000000,361060.000000


## EDA

1. How many records in total are there?
2. How many uniques values does each column contain?
3. How many missing (or null) values are there?
4. For numeric columns, what are its summary statistics
5. What are values appear most often in each column?
6. How are the values of each column distributed?

In [ ]:
# https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
df.count()

In [ ]:
df.createOrReplaceTempView("posts")

In [ ]:
# https://spark.apache.org/docs/latest/sql-ref-functions.html
spark.sql("SELECT DISTINCT ContentLicense FROM posts").show()

In [ ]:
spark.sql("SELECT count_if(Title IS NULL) FROM posts").show()

In [ ]:
df.select('Title', 'Body').show(6)

In [ ]:
df.na.drop(how='all').select('Title', 'Body').show(5)

In [ ]:
df.na.drop(subset=['Title']).select('Title', 'Body').show(5)

In [ ]:
df.na.fill({'Title': '# Needs Title', 'Body': ''}).select('Title', 'Body').show()

In [ ]:
df.sort(df.Score.desc()).limit(10).show()

In [ ]:
df.summary?

In [ ]:
df.describe(['AcceptedAnswerId', 'Score', 'ViewCount']).show()

In [ ]:
df.freqItems(['OwnerUserId', 'Tags'], support=.1).show(truncate=74, vertical=True)

In [ ]:
df.approxQuantile('Score', [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], .1)

In [ ]:
df.corr('Score', 'ViewCount')

In [ ]:
df.groupby('Score').count().show()

In [ ]:
df.groupby('Score').count().sort('Score').show()

In [ ]:
pandas_df = df.groupby('Score').count().sort('Score').toPandas()

In [ ]:
# do some scikit-learn or pytorch

In [ ]:
pandas_df.plot()

In [ ]:
pandas_df[pandas_df['Score'] < 30].plot()